In [12]:
# coding: utf-8
import sys
import torch, torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
#from torchsummary import summary
from torch.utils.data import TensorDataset, DataLoader

from tqdm import tqdm 
import numpy as np
from skimage.transform import resize
from numpy.fft import fftn, fftshift

import matplotlib.pyplot as plt
import matplotlib


import os

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from matplotlib import colors
import matplotlib.pyplot as plt

# from cupyx.scipy.signal import convolve2d as conv2
# import cupy as cp

# from scipy.signal import convolve2d as conv2np
import random

from IPython.display import clear_output

from multiprocessing import Pool, get_context
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from joblib import Parallel, delayed

from pathlib import Path

import h5py
import hdf5plugin

#from utils.preprocessing_functions import *

TypeError: 'module' object is not callable

In [ ]:
sys.path.append(Path("Y:/ptychosaxs/deconvolutionNN"))

NameError: name '__file__' is not defined

In [ ]:
plt.rcParams["image.cmap"] = "jet"

#reduce size of diffraction patterns
red=True

#save model pth
save=True

# Setting path
path = Path("Y:/ptychosaxs")  # /net/micdata/data2/12IDC mounted windows drive
#path=Path('/mnt/micdata2/12IDC/ptychosaxs') # micdata location 1 (refiner)
#path=Path('/net/micdata/data2/12IDC/ptychosaxs') # micdata location 2 (refiner and artemis)

# Join paths
MODEL_SAVE_PATH = path / 'trained_model/' # Automatically adds the correct separator
if (not os.path.isdir(MODEL_SAVE_PATH)):
    os.mkdir(MODEL_SAVE_PATH)
print(MODEL_SAVE_PATH)

In [ ]:
# Parameters
num = 13# 13, scanned with 400, subtracted out probe; 12, scanned with 400 lattice; 11, 288 lattice size (2x other scanned samples); 10, scanned with scaled by q^4; 9, scanned with no pinhole extra convolution; 8, scanned sample, pinhole extra convolution; 5, cindy; 4, chansong
data_location=path / f'data/diff_sim/{num}/'
print(data_location)
filenames=os.listdir(data_location)
filenames=[data_location / f for f in filenames]

# Combine into HDF5 (if not already done so)
output_file=path / f"data/combined_data_{num}.h5"
if output_file.exists():
    print(f'combined file {output_file} already exists')
else:
    print('loading and writing to h5')
    with h5py.File(output_file, "w") as h5f:
        for i, file_path in enumerate(filenames):
            data = np.load(file_path)
            h5f.create_dataset(f"convDP_{i}", data=data["convDP"])
            h5f.create_dataset(f"pinholeDP_{i}", data=data["pinholeDP"])

In [1]:
#load convoluted and ideal diffraction patterns
print('Loading data...')
# Load data with progress bar
with h5py.File(output_file, "r") as h5f:
    # Get the keys (dataset names)
    dataset_keys = list(h5f.keys()) 
    num_datasets = len(dataset_keys) // 2  # Assuming convDP and pinholeDP pairs
    print(f'{num_datasets} diffraction patterns')
    # Initialize empty lists for the data
    conv_DPs = []
    pinhole_DPs = []

    # Use tqdm for progress tracking
    for i in tqdm(range(num_datasets), desc="Loading HDF5 datasets"):
        conv_DPs.append(h5f[f"convDP_{i}"][:])  # Load convDP dataset
        pinhole_DPs.append(h5f[f"pinholeDP_{i}"][:])  # Load pinholeDP dataset

# convert to np arrays
conv_DPs=np.asarray(conv_DPs)
ideal_DPs=np.asarray(pinhole_DPs)


Loading data...


NameError: name 'h5py' is not defined

In [ ]:
probe_DPs=np.ones(conv_DPs.shape) #dummy array for testing network with a probe
#deal detector mask
#chansong
#conv_DPs=np.asarray([replace_2d_array_values_by_row_indices(replace_2d_array_values_by_row_indices(conv_DPs[i],ubound[0],ubound[1]),lbound[0],lbound[1]) for i in range(0,len(conv_DPs))])
#cindy
conv_DPs2=np.asarray([replace_2d_array_values_by_column_indices(replace_2d_array_values_by_column_indices(replace_2d_array_values_by_row_indices(replace_2d_array_values_by_row_indices(conv_DPs[i],0,8),247,255),0,8),247,255) for i in range(0,len(conv_DPs))])
